In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

scaler = StandardScaler()

#features = [f"V{x}" for x in range(1,2000)]
df = pd.read_csv('./Data/TCGAdata.txt', sep=" " ,header=0)
labels_df = pd.read_csv('./Data/TCGAlabels', sep=" " ,header=0)

#Standardizing the rows (transposing as fit_transform standardizes along columns)
scaled_df = pd.DataFrame(np.transpose(scaler.fit_transform(df.transpose())), columns=df.columns)

print(scaled_df)
print(labels_df)

            V1        V2        V3        V4        V5        V6        V7   
0    -0.919906  0.654030 -1.157337 -0.264374 -0.616969  2.418485  1.412586  \
1    -1.746010 -0.081994 -0.783148 -0.309184 -0.457870  0.507654  0.104541   
2    -0.600541 -0.190182 -1.011556 -0.067973 -0.238182  1.030204  0.703147   
3    -0.474599 -0.465606 -1.010290 -0.196135 -0.406030  0.286554 -1.244936   
4     0.900607  0.241042 -1.040232 -0.068804  0.240433 -0.638382  0.253359   
...        ...       ...       ...       ...       ...       ...       ...   
2882 -1.634421 -0.514738 -1.084321 -0.587557 -0.505288  1.604267 -1.993035   
2883  0.294937  0.249453 -0.934582  0.104487  0.637722  0.853234 -0.457945   
2884 -0.338485  0.668821 -1.248582 -0.328466  1.000972  2.275195 -0.529706   
2885 -0.445679 -0.153343 -0.226336 -0.302787  1.420491  1.050359 -2.311641   
2886 -0.223478  0.316791 -0.080916  0.249499  0.216114  0.331480 -0.854407   

            V8        V9       V10  ...     V1991     V1992    

In [55]:
# PCA
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  

# Example classifier, replace with your choice

# Assuming 'X' is your feature matrix and 'y' is your target variable

# Define a range of numbers of components to try
N=25
num_components_range = range(1, N)  # Example range, adjust as needed

# Initialize lists to store mean cross-validation scores for each number of components
SVC_mean_scores = np.zeros(N)
KNN_5_mean_scores = np.zeros(N)
KNN_500_mean_scores = np.zeros(N)
KNN_2000_mean_scores = np.zeros(N)

# Loop over different numbers of components
for n_components in tqdm(num_components_range):
    # Create a pipeline for classifiers
    SVC_pipeline = make_pipeline(PCA(n_components=n_components), SVC())  # Example pipeline, replace with your choice
    KNN_pipeline_5 = make_pipeline(PCA(n_components=n_components), KNeighborsClassifier(n_neighbors=5))
    KNN_pipeline_500 = make_pipeline(PCA(n_components=n_components), KNeighborsClassifier(n_neighbors=500))
    KNN_pipeline_2000 = make_pipeline(PCA(n_components=n_components), KNeighborsClassifier(n_neighbors=2000))

    #pca = PCA(n_components=n_components).fit_transform(scaled_df)
    #print(pca)
    #SVC_model = SVC().fit(pca, labels_df.values.ravel())
    #KNN_model = KNeighborsClassifier().fit(pca, labels_df.values.ravel())
    #print(cross_val_score(KNN_model, scaled_df, labels_df.values.ravel(), cv=5, scoring = 'accuracy' ).mean())
    
    # Mean cross-validation scores: 
    SVC_scores = cross_val_score(SVC_pipeline, scaled_df, labels_df.values.ravel(), cv=5, scoring = 'accuracy')
    SVC_mean_score = SVC_scores.mean()
    
    KNN_scores_5 = cross_val_score(KNN_pipeline_5, scaled_df, labels_df.values.ravel(), cv=5, scoring = 'accuracy')
    KNN_5_mean_score = KNN_scores_5.mean()

    KNN_scores_500 = cross_val_score(KNN_pipeline_500, scaled_df, labels_df.values.ravel(), cv=5, scoring = 'accuracy')
    KNN_500_mean_score = KNN_scores_500.mean()

    KNN_scores_2000 = cross_val_score(KNN_pipeline_2000, scaled_df, labels_df.values.ravel(), cv=5, scoring = 'accuracy')
    KNN_2000_mean_score = KNN_scores_2000.mean()
    
    SVC_mean_scores[n_components] = SVC_mean_score
    KNN_5_mean_scores[n_components] = KNN_5_mean_score
    KNN_500_mean_scores[n_components] = KNN_500_mean_score
    KNN_2000_mean_scores[n_components] = KNN_2000_mean_score

# Find the optimal number of components based on the highest mean cross-validation score
#SVC_optimal_n_components = SVC_mean_scores.index(max(SVC_mean_scores))+1
SVC_optimal_n_components = np.where(SVC_mean_scores==SVC_mean_scores.max())[0][0]+1
#KNN_optimal_n_components = KNN_mean_scores.index(max(KNN_mean_scores))+1
KNN_5_optimal_n_components = np.where(KNN_5_mean_scores==KNN_5_mean_scores.max())[0][0]+1

KNN_500_optimal_n_components = np.where(KNN_500_mean_scores==KNN_500_mean_scores.max())[0][0]+1

KNN_2000_optimal_n_components = np.where(KNN_2000_mean_scores==KNN_2000_mean_scores.max())[0][0]+1

# Print the optimal number of components
print("Optimal number of components for SVC:", SVC_optimal_n_components)
print("Optimal number of components for KNN (5 neighbours):", KNN_5_optimal_n_components)
print("Optimal number of components for KNN (500 neighbours):", KNN_500_optimal_n_components)
print("Optimal number of components for KNN (2000 neighbours):", KNN_2000_optimal_n_components)


  0%|          | 0/24 [00:00<?, ?it/s]

[[ 9.06364572]
 [12.39596925]
 [11.78465478]
 ...
 [ 3.31795413]
 [-6.69912907]
 [-0.67441426]]
0.9927245686734272


  0%|          | 0/24 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
SVC_mean_scores
#KNN_2000_mean_scores

array([0.        , 0.84862701, 0.95392587, 0.97540374, 0.98336941,
       0.9882191 , 0.98925956, 0.99133869, 0.99307059, 0.99307059,
       0.99272457, 0.99237795, 0.99272457, 0.99272457, 0.99272457,
       0.99341721, 0.99376383, 0.99411045, 0.99411045, 0.99445707,
       0.99376383, 0.99480309, 0.99410985, 0.99410985, 0.99376323])

In [ ]:
from sklearn.model_selection import cross_val_predict
clas = cross_val_score(SVC_pipeline, scaled_df, labels_df.values.ravel(), cv=5, scoring="accuracy")
#len(clas)-sum(clas == labels_df.values.ravel())
print(clas)


sum(labels_df.values.ravel() == 'BC')/len(labels_df)

[0.99653979 0.99653979 0.9982669  0.98960139 0.98786828]


0.42085209560096987